#### Cпособы найти упоминания товаров в отзывах

1. Составить правила для NE и извлечь их каким-нибудь инструментом типа yargy, только для английских текстов. Но здесь же и очевидные минусы - для датасета из миллионов отзывов невозможно будет идеально написать правила для каждой сущности, а также в отзывах редко можно встретить текст вида "модель + родовое понятие": автор отзыва скорее напишет просто название модели (если не просто использует местоимение), и предусмотреть такие случаи будет крайне сложно. Но если тот, кто пишет правила, сможет точно описать большинство случаев, на что могут уйти месяцы, то извлечение будет очень точным. Данные - сами отзывы.
2. Использовать готовую нейросеть для поиска именованных сущностей - например, LSTM-CRF. Минусы в том, что такой подход требует размеченных данных для обучения, но потенциально может показать хорошую точность.
3. Составление списка родовых понятий, по которым будут искаться NE, из ключевых слов, и расширение его с помощью векторных моделей, для последующего поиска n-грамм с этими понятиями. В этой работе мы попробуем именно такой способ. Данные - сами тексты и любые доступные метаданные, в нашем случае мы воспользуемся заголовками товаров. Минусы - скорее всего будет много шума, если не накладывать много условий на тип контекста, что уже ближе к подходу (1).

#### Импорт модулей

In [5]:
import re
import nltk
import json
import RAKE
import spacy
import gensim
import pandas as pd
from nltk.metrics import *
from nltk.collocations import *
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
from tqdm.notebook import tqdm
from collections import Counter

#### 0. Чтение файлов

In [6]:
def read(filename, n=None):
    with open(filename) as f:
        dict_train = f.readlines()[:n]
    dataset = []
    for d in tqdm(dict_train):
        dataset.append(json.loads(d))
    dataset = pd.DataFrame(dataset)
    return dataset

In [7]:
reviews = read('Cell_Phones_and_Accessories.json', 300000)
metadata = read('meta_Cell_Phones_and_Accessories.json')

  0%|          | 0/300000 [00:00<?, ?it/s]

  0%|          | 0/590071 [00:00<?, ?it/s]

In [8]:
reviews = reviews.drop(['overall', 'verified', 'reviewTime', 
                        'reviewerID', 'reviewerName', 'summary', 
                        'unixReviewTime', 'vote', 'image', 
                        'style'], axis=1)

In [9]:
metadata = metadata.drop(['tech1', 'fit', 'also_buy', 
                          'tech2', 'feature', 'main_cat',
                          'description', 'brand', 'category',
                          'rank', 'also_view', 'details', 
                          'similar_item', 'date', 'price', 
                          'imageURL', 'imageURLHighRes'], axis=1)

In [10]:
result = pd.merge(reviews, metadata, on='asin', how='inner')
result = result[['asin', 'title', 'reviewText']]

In [11]:
result.head()

,asin,title,reviewText
0,7391002801,Silver Elegant Butterfly Foot Ankle Chain Summ...,Beautiful item; received timely. Thank you.
1,7391002801,Silver Elegant Butterfly Foot Ankle Chain Summ...,Had this for 2 weeks. Had to replace screen p...
2,7391002801,Silver Elegant Butterfly Foot Ankle Chain Summ...,The apple is not centered in the hole on the b...
3,7391002801,Silver Elegant Butterfly Foot Ankle Chain Summ...,Case is cheaply made. If you aren't using an a...
4,7391002801,Silver Elegant Butterfly Foot Ankle Chain Summ...,This case is a really good thing. When you're ...


#### 1. Препроцессинг

In [15]:
nlp = spacy.load('en_core_web_sm')
stop_words = stopwords.words('english')
rake = RAKE.Rake(stop_words)
lemmatizer = WordNetLemmatizer()

In [20]:
def preprocess(sentence):
    res = []
    if isinstance(sentence, str):
        sentence = sentence.translate(str.maketrans('', '', punctuation))
        tokens = nltk.word_tokenize(sentence)
        for token in tokens:
            token = token.lower()
            if token not in stop_words:
                res.append(lemmatizer.lemmatize(token))
    return ' '.join(res)

In [21]:
tqdm.pandas()
result['reviewNormalized'] = result['reviewText'].progress_apply(preprocess)

  0%|          | 0/306337 [00:00<?, ?it/s]

In [22]:
result.head()

,asin,title,reviewText,reviewNormalized
0,7391002801,Silver Elegant Butterfly Foot Ankle Chain Summ...,Beautiful item; received timely. Thank you.,beautiful item received timely thank
1,7391002801,Silver Elegant Butterfly Foot Ankle Chain Summ...,Had this for 2 weeks. Had to replace screen p...,2 week replace screen protector outer ring sna...
2,7391002801,Silver Elegant Butterfly Foot Ankle Chain Summ...,The apple is not centered in the hole on the b...,apple centered hole back fit iphone 8 plus pro...
3,7391002801,Silver Elegant Butterfly Foot Ankle Chain Summ...,Case is cheaply made. If you aren't using an a...,case cheaply made arent using apple charger fi...
4,7391002801,Silver Elegant Butterfly Foot Ankle Chain Summ...,This case is a really good thing. When you're ...,case really good thing youre tired bright mass...


#### 2. Извлечение ключевых слов

In [23]:
def get_kw(col):
    keywords = []
    for i, row in tqdm(result.iterrows()):
        try:
            kw = rake.run(result[col][i], maxWords=3, minFrequency=1)
            for k in kw:
                keywords.append(k[0])
        except:
            pass
    return keywords

##### 2.1. Из текстов

In [25]:
from_texts = get_kw('reviewText')
Counter(from_texts).most_common(50)

0it [00:00, ?it/s]

[('phone', 76139),
 ('use', 44572),
 ('case', 33052),
 ('get', 31329),
 ('one', 30767),
 ('product', 28864),
 ('bought', 27008),
 ('like', 24688),
 ('work', 23640),
 ('well', 22665),
 ('time', 22438),
 ('good', 22234),
 ('easy', 21864),
 ('great', 20902),
 ('price', 20891),
 ('love', 20484),
 ('would', 19475),
 ('got', 19472),
 ('used', 17576),
 ('charge', 16148),
 ('buy', 16116),
 ('need', 15891),
 ('using', 15370),
 ('iphone', 15355),
 ('also', 15149),
 ('put', 14089),
 ('problem', 13672),
 ('fit', 13655),
 ('however', 13337),
 ('want', 13306),
 ('screen', 13251),
 ('way', 12614),
 ('back', 12530),
 ('far', 12329),
 ('battery', 12212),
 ('purchased', 12145),
 ('know', 11947),
 ('works', 11854),
 ('money', 11849),
 ('think', 11661),
 ('much', 11459),
 ('able', 11174),
 ('lot', 11069),
 ('even', 10911),
 ('find', 10684),
 ('happy', 10669),
 ('see', 10330),
 ('take', 10165),
 ('amazon', 10159),
 ('found', 10101)]

##### 2.2. Из заголовков товаров

In [26]:
from_titles = get_kw('title')
Counter(from_titles).most_common(50)

0it [00:00, ?it/s]

[('black', 55989),
 ('retail packaging', 17776),
 ('&amp', 15653),
 ('warranty', 11145),
 ('white', 10968),
 ('manufacturer', 10612),
 ('discontinued', 10108),
 ('3', 8612),
 ('iphone 3g', 7720),
 ('iphone 4', 7324),
 ('3gs', 6889),
 ('smartphones', 6618),
 ('generic 3 pack', 6360),
 ('fits', 6320),
 ('non-retail packaging', 6079),
 ('3g', 5404),
 ('iphone', 5340),
 ('gps', 5143),
 ('htc evo 4g', 5006),
 ('silver', 4917),
 ('version', 4666),
 ('2 mp camera', 4643),
 ('1 pack', 4468),
 ('camera', 4423),
 ('w/built-', 4278),
 ('[retail packaging]', 4183),
 ('otterbox defender case', 4174),
 ('wi-fi', 3757),
 ('new trent', 3594),
 ('laser + flashlight', 3594),
 ('office', 3477),
 ('blue', 3411),
 ('clear', 3343),
 ('ipod', 3241),
 ('sprint', 3182),
 ('charging cable', 3063),
 ('wilson electronics', 2973),
 ('small home', 2902),
 ('dt', 2902),
 ('apple iphone 3g', 2816),
 ('6ft', 2693),
 ('case', 2536),
 ('2', 2499),
 ('apple iphone 4', 2418),
 ('compatible', 2374),
 ('t-mobile', 2308),
 (

Отберём из выделенных выше ключевых слов наиболее подходящие для извлечения NE:

In [27]:
entities = ['phone', 'case', 'iphone', 'smartphone', 'htc', 'apple', 'headset', 'cover', 'charger']

С помощью word2vec расширим этот список близкими к ним словами:

In [28]:
model = gensim.models.Word2Vec(result.reviewNormalized.apply(lambda x: x.split()), window=2)

In [29]:
extension = []
for entity in entities:
    for word in model.wv.most_similar(entity, topn=3):
        extension.append(word[0])
entities.extend(extension)

In [30]:
entities

['phone',
 'case',
 'iphone',
 'smartphone',
 'htc',
 'apple',
 'headset',
 'cover',
 'charger',
 'cellphone',
 'iphone',
 'device',
 'cover',
 'skin',
 'sleeve',
 'iphone4',
 'iphones',
 'phone',
 'pda',
 'smartphones',
 'smart',
 'phonehtc',
 'env',
 '4g',
 'verizon',
 'att',
 'sprint',
 'earpiece',
 'headpiece',
 'headphone',
 'case',
 'covering',
 'casing',
 'recharger',
 'adapter',
 'wart']

word2vec добавил лишних имён в список, почистим его:

In [48]:
to_remove = ['phonehtc', 'recharger', 'cellphone', 'pda', 'verizon', 'env',
             'att', 'smartphones', 'iphones', 'smart', 'wart', 'iphone4', 'headpiece',
             'sleeve', 'covering', 'casing']
entities = [entity for entity in entities if entity not in to_remove]

В итоге имеется следующий список родовых понятий, для которых мы будем извлекать биграммы:

In [49]:
entities

['phone',
 'case',
 'iphone',
 'smartphone',
 'htc',
 'apple',
 'headset',
 'cover',
 'charger',
 'iphone',
 'device',
 'cover',
 'skin',
 'phone',
 '4g',
 'sprint',
 'earpiece',
 'headphone',
 'case',
 'adapter']

#### 3. Извлечение биграмм

In [50]:
def n_gram(text, mode):
    text = text.split()
    res = []
    for w in text:
        if w in entities:
            n = text.index(w)
            try:
                if mode == 'left':
                    res.append((text[n-1], text[n]))
                elif mode == 'right':
                    res.append((text[n], text[n+1]))
            except IndexError:
                continue
    return res

In [51]:
n_grams = []
for i, row in result.iterrows():
    left_context = n_gram(result['reviewNormalized'][i], 'left')
    right_context = n_gram(result['reviewNormalized'][i], 'right')
    n_grams.extend(left_context)
    n_grams.extend(right_context)

In [52]:
bigrams_counter = Counter(n_grams)
bigrams_counter.most_common(10)

[(('cell', 'phone'), 19532),
 (('phone', 'case'), 11744),
 (('bluetooth', 'headset'), 10439),
 (('iphone', '4'), 8821),
 (('bought', 'phone'), 7089),
 (('case', 'iphone'), 5956),
 (('fit', 'phone'), 4595),
 (('got', 'phone'), 4534),
 (('case', 'phone'), 4476),
 (('case', 'fit'), 4169)]

#### 4. Ранжирование биграмм

In [53]:
bm = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(n_grams)
finder.apply_ngram_filter(lambda x, y: y in entities)
finder.apply_freq_filter(20)

##### 4.1. Jaccard

In [54]:
jaccard = finder.nbest(bm.jaccard, 10000)
jaccard[:15]

[('htc', 'evo'),
 ('iphone', '4'),
 ('headphone', 'jack'),
 ('apple', 'store'),
 ('sprint', 'store'),
 ('htc', 'incredible'),
 ('iphone', '3gs'),
 ('apple', 'product'),
 ('iphone', '3g'),
 ('4g', 'lte'),
 ('charger', 'work'),
 ('htc', 'touch'),
 ('apple', 'logo'),
 ('iphone', '5'),
 ('case', 'fit')]

##### 4.2. PMI

In [55]:
pmi = finder.nbest(bm.pmi, 10000)
pmi[:15]

[('4g', 'lte'),
 ('htc', 'tytn'),
 ('htc', 'aria'),
 ('htc', 'sensation'),
 ('htc', 'hd2'),
 ('htc', 'desire'),
 ('htc', 'thunderbolt'),
 ('htc', 'hero'),
 ('htc', 'wildfire'),
 ('htc', 'inspire'),
 ('apple', 'logo'),
 ('htc', 'fuze'),
 ('htc', 'incredible'),
 ('sprint', 'network'),
 ('htc', 'evo')]

##### 4.3. Likelihood ratio

In [56]:
likelihood_ratio = finder.nbest(bm.likelihood_ratio, 10000)
likelihood_ratio[:15]

[('iphone', '4'),
 ('iphone', '3gs'),
 ('iphone', '3g'),
 ('headphone', 'jack'),
 ('htc', 'evo'),
 ('phone', 'cell'),
 ('iphone', '5'),
 ('apple', 'store'),
 ('phone', 'bluetooth'),
 ('case', 'cell'),
 ('phone', 'call'),
 ('case', 'fit'),
 ('charger', 'work'),
 ('case', 'bluetooth'),
 ('phone', 'bought')]

#### 5. Группировка коллокаций

В общем и целом все метрики показали хороший результат, но самой подходящей мне кажется Jaccard - она вывела вперёд что-то наиболее близкое к именованиям товаров.

In [57]:
results = {}
for entity in entities:
    results[entity] = []
    for element in jaccard:
        if (element[0] == entity or element[1] == entity) and len(results[entity]) < 5:
            results[entity].append(' '.join(element))

In [58]:
for k, v in results.items():
    print(k, '\n---')
    for ne in v: print(ne)
    print('\n')

phone 
---
phone work
phone great
phone well
phone would
phone one


case 
---
case fit
case great
case look
case ive
case one


iphone 
---
iphone 4
iphone 3gs
iphone 3g
iphone 5
iphone 6


smartphone 
---
smartphone market
smartphone user
smartphone im
smartphone feature
smartphone ever


htc 
---
htc evo
htc incredible
htc touch
htc droid
htc hero


apple 
---
apple store
apple product
apple logo
apple bumper
apple cable


headset 
---
headset work
headset ive
headset one
headset use
headset great


cover 
---
cover fit
cover screen
cover back
cover great
cover front


charger 
---
charger work
charger car
charger charge
charger came
charger cable


device 
---
device work
device would
device use
device one
device like


skin 
---
skin fit
skin blackberry
skin jelly
skin screen
skin look


4g 
---
4g lte
4g signal
4g touch
4g perfectly
4g evo


sprint 
---
sprint store
sprint pc
sprint network
sprint service
sprint customer


earpiece 
---
earpiece comfortable
earpiece volume
earpie

Ожидаемо названия моделей там, где это стоит упоминания - рядом с торговыми марками, а в более общих родовых понятиях как "адаптер" или "зарядка" вместо названия моделей общие характеристики типа "charger car" - NE, которая, видимо, потеряля предлог во время препроцессинга (возможно, он был здесь лишним).